In [1]:
# importing necessary libraries

import nltk, re, string
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pickle

In [2]:
# Preprocessing of the tweets that is our data

def process_tweet(tweet):
    stemmer = nltk.PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [3]:
# This is the most important part of the whole code: 
# The reason is our feature set through which we will be training our model on will be build here.

In [4]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [5]:
# Checking how the above code works with an example.

tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
res = build_freqs(tweets, ys)
print(res)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}


In [6]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [7]:
# split the data into two pieces, one for training and one for testing.
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

In [8]:
train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [9]:
# combine positive and negative labels
# We are building our y - target variable here
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [10]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

In [11]:
# check out the output

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11337


In [12]:
# test the function below

print('This is an example of a positive tweet: \n', train_x[20])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[20]))

This is an example of a positive tweet: 
 #FollowFriday @MBandScott_ @Eric_FLE @pointsolutions3 for being top new followers in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'new', 'follow', 'commun', 'week', ':)']


In [13]:
#  Extracting the features

def extract_features(tweet, freqs):
    """
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    """

    word_l = process_tweet(tweet)
    x = np.zeros((1, 3))

    # bias term is set to 1
    x[0, 0] = 1

    for word in word_l:
        # increment the word count for the positive label 1
        x[0, 1] += freqs.get((word, 1.0), 0)
        # increment the word count for the negative label 0
        x[0, 2] += freqs.get((word, 0.0), 0)

    assert (x.shape == (1, 3))
    return x

In [14]:
train_x[22]

'@gculloty87 Yeah I suppose she was lol! Chat in a bit just off out x :))'

In [15]:
# test on training data

tmp1 = extract_features(train_x[22], freqs)
print(tmp1)

[[1.000e+00 3.006e+03 1.240e+02]]


In [16]:
# Try to understand what all these three numbers mean. 
# Usually we get a dataset with a lot of features/columns, here we just have text data.
# Those three numbers are the feature set that we have build using build_freq() and extract_features() function.
# build_freq() builds a dictionary having words as keys and the number of times they have occurred in corpus as values.
# Extract feature takes in sum of these values for positive and negative words, i.e. tmp1[1] and tmp[2]


In [17]:
# How these features will be used to predict in Logistic Regression

# First a hypothesis is build which for our case will be h(x) = b1 + b2*x1 + b3*x2
# here b1 = 1, b2 and b3 are determined by cost and gradient function, x1 and x2 are the positive and negative words feature set.

In [18]:
# Training Your Model

# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

In [19]:
# Importing logistic regression and train_test_split from scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [20]:
# Assume you have your data in X (features) and y (target variable)
# X_train, X_test, y_train, y_test will be your training and testing sets

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
# Creating a logistic regression model
model = LogisticRegression()

In [22]:
# Training the model on the training data
model.fit(X_train, y_train)

C:\Users\USER\anaconda3\envs\DataFrame\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [23]:
# Making predictions on the testing data
y_pred = model.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score

# Assuming 'predictions' are the predictions made by your model and 'y_test' is the actual target values
accuracy = accuracy_score(y_test, y_pred)

print(f'Model Accuracy: {accuracy * 100:.2f}%')

Model Accuracy: 98.62%


In [25]:
model.predict(X_test[56].reshape(1, -1))

array([1.])

In [26]:
texts = "@gculloty87 Yeah I suppose she was lol! I hate you and love ## 2 you with all ##$ my heart *&"
text1 = extract_features(texts, freqs)
text1

array([[  1., 476., 298.]])

In [27]:
model.predict(text1)

array([1.])

In [28]:
# Predict with your own tweet

def pre(texts):
    text1 = extract_features(texts, freqs)
    yhat = model.predict(text1)
    if yhat > 0.5:
        return 'Positive sentiment'
    elif yhat == 0:
        return 'Neutral sentiment'
    else:
        return 'Negative sentiment'

In [29]:
my_tweet = "Sitting in traffic for hours. Just what I needed to make my evening perfect. 🚗🤬 #TrafficJam #LateAgain"

res = pre(my_tweet)
print(res)

Positive sentiment


In [30]:
# save the model to disk

# filename = 'pickle'
# pickle.dump(model, open(filename, 'wb'))

with open(r'C:\Users\USER\Python Files\Sentiment.pickle', "wb") as b: 
    pickle.dump(model, b)